# Fine Tuning notebook

In [1]:
#%pip install pandas numpy torcheval torch matplotlib tensorboard torchvision=0.14.1
#%pip uninstall -y torchvision
# %pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
# %pip install opencv-python

import getpass


from pathlib import Path
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2

import torch

from torch import nn
from torch import optim

import torch.nn.functional as F
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import get_model, get_model_weights, get_weight, list_models


import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import random_split, ConcatDataset

# from transformers import ViTFeatureExtractor


/apps/pytorch/2.0.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# hyper params
batch_size = 120
learning_rate = 0.001
epochs = 15

In [3]:
!pwd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}\n')
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

/orange/ufdatastudios/christan/mixteclabeling/notebooks
Using device: cuda


NVIDIA A100-SXM4-80GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Define path to images

In [4]:
basepath = Path(f'/home/{getpass.getuser()}/toorange/alexwebber/mixteclabeling') # Base data directory
path_v = basepath / 'data/labeled_figures/codex_vindobonensis/gender/'
path_n = basepath / 'data/labeled_figures/codex_nuttall/gender/'
path_s = basepath / 'data/labeled_figures/codex_selden/gender/'

In [5]:
## Define image transforms
## List of transforms https://pytorch.org/vision/stable/auto_examples/plot_transforms.html
transform = transforms.Compose(
    [transforms.ToTensor(),
     # AddRandomBlockNoise(),
     transforms.Resize((227, 227), antialias=True),
     # transforms.Grayscale(),
     
     #transforms.ColorJitter(contrast=0.5),
     #transforms.RandomRotation(360),     # Maybe useful for standng and sitting
     #transforms.RandomHorizontalFlip(50),
     #transforms.RandomVerticalFlip(50)
])

# transform = transforms.Compose([transforms.Resize((224, 224)), 
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


## Load images into PyTorch dataset
vindobonensis_dataset = datasets.ImageFolder(path_v, transform=transform)
nuttall_dataset = datasets.ImageFolder(path_n, transform=transform)
selden_dataset = datasets.ImageFolder(path_s, transform=transform)

figures_dataset = ConcatDataset([vindobonensis_dataset, nuttall_dataset, selden_dataset])

In [6]:
import datetime
def printdate(dt=datetime.datetime.now()):
    """print a date and time string containing only numbers and dashes"""

    # your code here
    if dt.hour < 10:
        hour = '0' + str(dt.hour)
    else:
        hour = str(dt.hour)

    if dt.minute < 10:
        minute = '0' + str(dt.minute)
    else:
        minute = str(dt.minute)

    d = '{}-{}-{}-{}-{}'.format(str(dt.month),str(dt.day),str(dt.year),hour,minute)
    print(d)
    return d
    
writer = SummaryWriter(log_dir=f'tmp/{printdate()}')

6-11-2023-18-46


2023-06-11 18:46:28.520203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 18:46:29.937270: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
train_set, validation_set, test_set = random_split(figures_dataset, [0.6, 0.2, 0.2])

print(f"{len(train_set)}, {len(validation_set)}, {len(test_set)}")

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True)

validate_loader = torch.utils.data.DataLoader(validation_set, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size,  shuffle = True)

412, 137, 137


In [8]:
# model = models.vgg16(pretrained=True)
model = get_model("vgg19", weights=torchvision.models.VGG19_Weights.DEFAULT)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [9]:
# Freeze pretrained model parameters to avoid backpropogating through them
for parameter in model.parameters():
    parameter.requires_grad = False


from collections import OrderedDict

# Build custom classifier
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                        ('relu', nn.ReLU()),
                                        ('drop', nn.Dropout(p=0.5)),
                                        ('fc2', nn.Linear(5000, 102)),
                                        ('output', nn.LogSoftmax(dim=1))]))

model.classifier = classifier

In [10]:
# Function for the validation pass
def validation(model, validateloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validateloader):

        images, labels = images.to(device), labels.to(device)

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)
        
        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return val_loss, accuracy

In [11]:
# Loss function and gradient descent

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

In [12]:
# Train the classifier

def train_classifier():

    steps = 0
    print_every = 5

    model.to(device)

    for e in range(epochs):

        model.train()

        running_loss = 0

        for images, labels in iter(train_loader):

            steps += 1

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:

                model.eval()

                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    validation_loss, accuracy = validation(model, validate_loader, criterion)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Train Loss: {:.6f}.. ".format(running_loss/print_every),
                      "Val Loss: {:.6f}.. ".format(validation_loss/len(validate_loader)),
                      "Val Accuracy: {:.6f}".format(accuracy/len(validate_loader)))


                running_loss = 0
                model.train()
                    
train_classifier()                    

Epoch: 2/15..  Train Loss: 0.312921..  Val Loss: 0.870329..  Val Accuracy: 0.900000
Epoch: 3/15..  Train Loss: 0.302420..  Val Loss: 1.768469..  Val Accuracy: 0.887010
Epoch: 4/15..  Train Loss: 0.219616..  Val Loss: 1.437138..  Val Accuracy: 0.832598
Epoch: 5/15..  Train Loss: 0.052049..  Val Loss: 2.258097..  Val Accuracy: 0.882843
Epoch: 7/15..  Train Loss: 0.000284..  Val Loss: 0.610623..  Val Accuracy: 0.933333
Epoch: 8/15..  Train Loss: 0.000600..  Val Loss: 0.750802..  Val Accuracy: 0.924755
Epoch: 9/15..  Train Loss: 0.000178..  Val Loss: 0.706828..  Val Accuracy: 0.962500
Epoch: 10/15..  Train Loss: 0.010645..  Val Loss: 0.929518..  Val Accuracy: 0.907843
Epoch: 12/15..  Train Loss: 0.001681..  Val Loss: 0.731031..  Val Accuracy: 0.962500
Epoch: 13/15..  Train Loss: 0.000005..  Val Loss: 0.848013..  Val Accuracy: 0.903677
Epoch: 14/15..  Train Loss: 0.000039..  Val Loss: 0.719313..  Val Accuracy: 0.937500
Epoch: 15/15..  Train Loss: 0.000057..  Val Loss: 1.800655..  Val Accura

In [13]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to(device)

    with torch.no_grad():
    
        accuracy = 0
    
        for images, labels in iter(test_loader):
    
            images, labels = images.to(device), labels.to(device)
    
            output = model.forward(images)

            probabilities = torch.exp(output)
        
            equality = (labels.data == probabilities.max(dim=1)[1])
        
            accuracy += equality.type(torch.FloatTensor).mean()
        
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))    
        
        
test_accuracy(model, test_loader)

Test Accuracy: 0.9624999761581421


In [14]:
# Save the checkpoint

def save_checkpoint(model):

    model.class_to_idx = {0: "female", 1: "male",
                                        1.0: "male"}

    checkpoint = {'arch': "vgg16",
                  'class_to_idx': model.class_to_idx,
                  'model_state_dict': model.state_dict()
                 }

    torch.save(checkpoint, 'checkpoint.pth')
    
save_checkpoint(model)    

In [15]:
from collections import OrderedDict

# Function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    
    if checkpoint['arch'] == 'vgg16':
        
        model = models.vgg16(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Architecture not recognized.")
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                            ('relu', nn.ReLU()),
                                            ('drop', nn.Dropout(p=0.5)),
                                            ('fc2', nn.Linear(5000, 102)),
                                            ('output', nn.LogSoftmax(dim=1))]))

    model.classifier = classifier
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model

#model = load_checkpoint('checkpoint.pth')
#print(model)

In [16]:
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # Process a PIL image for use in a PyTorch model
    
    pil_image = Image.open(image_path)
    
    # Resize
    if pil_image.size[0] > pil_image.size[1]:
        pil_image.thumbnail((5000, 256))
    else:
        pil_image.thumbnail((256, 5000))
        
    # Crop 
    left_margin = (pil_image.width-224)/2
    bottom_margin = (pil_image.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    
    pil_image = pil_image.crop((left_margin, bottom_margin, right_margin, top_margin))
    
    # Normalize
    np_image = np.array(pil_image)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    # np_image = (np_image - mean) / std
    
    # PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array
    # Color channel needs to be first; retain the order of the other two dimensions.
    np_image = np_image.transpose((2, 0, 1))
    
    return np_image

In [17]:
# Implement the code to predict the class from an image file

def predict(image_path, model, topk=2):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # image = process_image(image_path)
    imageset = datasets.ImageFolder(image_path, transform=transform)
    dataloader = torch.utils.data.DataLoader(imageset, batch_size = batch_size, shuffle = True)
    
#     # Convert image to PyTorch tensor first
#     image = torch.from_numpy(image).type(torch.cuda.FloatTensor)
#     print(image.shape)
#     print(type(image))
    
#     # Returns a new tensor with a dimension of size one inserted at the specified position.
#     image = image.unsqueeze(0)


    data_iter = iter(test_loader)
    # next(data_iter) # uncomment to get another test ietm
    image, label = next(data_iter) #get the first test item
    
    output = model.forward(image.to(device))
    print(f"{output.shape=}")
    
    probabilities = torch.exp(output)
    
    # Probabilities and the indices of those probabilities corresponding to the classes
    top_probabilities, top_indices = probabilities.topk(topk)
    
    # Convert to lists
    top_probabilities = top_probabilities.detach().type(torch.FloatTensor).numpy().tolist()[0] 
    top_indices = top_indices.detach().type(torch.FloatTensor).numpy().tolist()[0] 
    
    # Convert topk_indices to the actual class labels using class_to_idx
    # Invert the dictionary so you get a mapping from index to class.
    
    idx_to_class = {value: key for key, value in model.class_to_idx.items()}
    print(model.class_to_idx.items())
    print(idx_to_class)
    print(top_indices)
    
    # top_classes = [idx_to_class[int(index)] for index in top_indices]
    
    # return top_probabilities, top_classes
    return top_probabilities, ['female', 'male']
    

# probs, classes = predict('/orange/ufdatastudios/christan/mixteclabeling/044-a-03.png', model)
probs, classes = predict('/orange/ufdatastudios/christan/mixteclabeling/notebooks/imgs', model)

print(probs)
print(classes)

output.shape=torch.Size([120, 102])
dict_items([(0, 'female'), (1, 'male')])
{'female': 0, 'male': 1}
[1.0, 0.0]
[1.0, 6.728262391675344e-11]
['female', 'male']


In [18]:
#from torchvision.models import get_model, get_model_weights, get_weight, list_models
list_models()
mod = torchvision.models.VGG19_Weights
mod

<enum 'VGG19_Weights'>